In [1]:
import pandas as pd
import sys
sys.path.append('../')

from TS_model.feature_engeneering import FeatureEngineer

%load_ext autoreload
%autoreload 2

In [2]:
# # пересчет баланса 
# data = pd.read_excel('../Project 2_2023.xlsx', parse_dates=True, index_col=0)
# data.columns = data.columns.str.lower()
# data['balance'] = data['income'] - data['outcome']
# data.to_excel('../Project 2_2023.xlsx')

In [3]:
# добавление фичей 
data = pd.read_csv('../Data/final.csv', parse_dates=['date'])
data.columns = data.columns.str.lower()
data['date'] = pd.to_datetime(data['date'])

macro_cols = [
    'digitalization_level', 'age_0_14', 'age_15_64', 'age_65_plus',
    'population_density', 'consumer_confidence_index', 'gdp_per_capita',
    'unemployment_rate', 'trade_balance', 'inflation_rate', 'close', 'usd_rate'
]
macro_df = data.set_index('date')[macro_cols]

In [4]:
# добавление налоговых дат 
tax_dates_dict = {
    "2017-01-25": ["НДФЛ", "НДС"],
    "2017-02-15": ["Страховые взносы"],
    "2017-04-28": ["НДФЛ", "НДС"],
    "2017-05-01": ["Акцизы"],
    "2017-10-10": ["Налог на имущество организаций"],
    "2018-01-25": ["НДФЛ", "НДС"],
    "2018-02-15": ["Налог на землю"],
    "2019-04-25": ["НДС"],
    "2019-07-01": ["Транспортный налог"],
    "2020-07-15": ["НДФЛ"],
    "2021-03-15": ["НДФЛ", "Страховые взносы", "Налог на прибыль"],
}


tax_dates = list(tax_dates_dict.keys())
tax_dates = pd.to_datetime(tax_dates)

In [5]:
feature_extactor = FeatureEngineer(data, date_col='date')

In [6]:
feature_extactor = (
    feature_extactor.add_lag_features(lags=[1, 2, 3, 7])
      .add_rolling_features(windows=[3, 7, 30])
      .add_seasonal_features()
      .add_macro_features(macro_df)
)


feature_extactor = feature_extactor.add_special_dates(tax_dates)

/Users/legornevas/Desktop/TS_project/TS_project/notebooks/../TS_model/feature_engeneering.py:82: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  self.features.fillna(method='ffill', inplace=True)


In [7]:
x, y = feature_extactor.get_feature_df()

In [8]:
data = pd.concat([x, y], axis=1)
data.to_excel('../Data/full_features_data.xlsx')